In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

In [2]:
sampleRawDF = returnCollectedExistingData(
    benchmarkNames=["cg"],
    classes=["A", "B", "C", "D", "E", "F"],
    processes=[2, 4, 8, 16, 32, 64, 128, 256],
    csvDirPath=csvDirPath,
)
sampleRawDF

test_addInitDataToRawDF()

,functionName,functionCallNum,benchmarkName,benchmarkClass,process,nx_default,ny_default,nz_default,nit_default
0,functionAtMG,32,mg,F,64,4096,4096,4096,50
